In [ ]:
from __init__ import *

In [ ]:
directory_path = "{}feature/".format(SETTING["directory_path"])

kraft.path.make(directory_path)

In [ ]:
for url in ("https://ndownloader.figshare.com/files/22629137",):

    kraft.internet.download(
        url, directory_path, name="sample_info.csv", overwrite=SETTING["overwrite"]
    )

## Map cell line name

In [ ]:
name_to_rename = {}

In [ ]:
name_to_id = (
    pd.read_csv(
        "../input/cell-line-synon-names.txt",
        sep="\t",
        usecols=(0, 1),
        index_col=1,
        squeeze=True,
    )
    .dropna()
    .to_dict()
)

id_to_rename = (
    pd.read_csv(
        "../input/cell-line-best-names.txt",
        sep="\t",
        usecols=(0, 1),
        index_col=0,
        squeeze=True,
    )
    .dropna()
    .to_dict()
)

for name, id_ in name_to_id.items():

    name_to_rename[name.lower()] = id_to_rename[id_]

len(name_to_rename)

In [ ]:
for name, rename in (
    pd.read_csv(
        "../input/ice000.cols.cell_line_identity.2777.csv",
        usecols=(2, 3),
        index_col=0,
        squeeze=True,
    )
    .dropna()
    .items()
):

    name_lower = name.lower()

    if name_lower in name_to_rename:

        rename_now = name_to_rename[name_lower]

        if rename_now != rename:

            print("{}: {} => {}".format(name, rename_now, rename))

    name_to_rename[name_lower] = rename

len(name_to_rename)

In [ ]:
feature_x_sample = pd.read_csv(
    "{}sample_info.csv".format(directory_path), index_col=1
).T

In [ ]:
for rename, names in feature_x_sample.iloc[:3, :].items():

    for name in names.dropna():

        name_lower = name.lower()

        if name_lower in name_to_rename:

            rename_now = name_to_rename[name_lower]

            if rename_now != rename:

                print("{}: {} => {}".format(name, rename_now, rename))

        name_to_rename[name_lower] = rename

len(name_to_rename)

In [ ]:
for rename in tuple(name_to_rename.values()):

    name_to_rename[rename.lower()] = rename

len(name_to_rename)

In [ ]:
name_to_rename = pd.Series(data=name_to_rename).sort_values()

name_to_rename.index.name = "Name"

name_to_rename.name = "Rename"

name_to_rename.to_csv(
    "{}cell_line_name_rename.tsv".format(directory_path), sep="\t", header=True
)

name_to_rename

## Make feature_x_sample

### All

In [ ]:
feature_x_sample.loc["Is Fibroblast", :] = tuple(
    int(bool(re.match(".*FIBROBLAST$", name)))
    for name in feature_x_sample.loc["CCLE_Name", :].to_numpy()
)

feature_x_sample.index.name = "Feature"

feature_x_sample.columns.name = "Cell Line"

kraft.pd.error_axes(feature_x_sample)

feature_x_sample.to_csv("{}feature_x_sample.tsv".format(directory_path), sep="\t")

kraft.pd.count_value(feature_x_sample.T)

feature_x_sample

### Continuous

In [ ]:
continuous_feature_x_sample = feature_x_sample.loc[["age"]]

continuous_feature_x_sample.to_csv(
    "{}continuous_feature_x_sample.tsv".format(directory_path), sep="\t"
)

continuous_feature_x_sample

### Binary

In [ ]:
df_ = [feature_x_sample.loc[["Is Fibroblast"], :]]

for label in (
    "sex",
    "sample_collection_site",
    "primary_or_metastasis",
    "primary_disease",
    "Subtype",
    "lineage",
    "lineage_subtype",
    "lineage_sub_subtype",
    "lineage_molecular_subtype",
):

    df = kraft.pd.binarize(feature_x_sample.loc[label, :])

    df.index = ("({}) {}".format(label, label_) for label_ in df.index.to_numpy())

    df_.append(df)

binary_feature_x_sample = pd.concat(df_)

binary_feature_x_sample.index.name = "Feature"

binary_feature_x_sample.to_csv(
    "{}binary_feature_x_sample.tsv".format(directory_path), sep="\t"
)

binary_feature_x_sample